In [0]:
from slack_sdk import WebClient
import pyspark.pandas as ps
import matplotlib.pyplot as plt
import os

spark.conf.set("spark.sql.ansi.enabled", False)

In [0]:
slack_token = '<slack-token>'
client = WebClient(token=slack_token)

In [0]:
file_name = dbutils.fs.ls("dbfs:/databricks-results/prata/valores_reais")[-1].name

path = "/dbfs/databricks-results/prata/valores_reais/" + file_name

In [0]:
send_file_csv = client.files_upload_v2(
    channel="C096LMP8UJE",
    title="Arquivo no formato CSV do valor do real convertido",
    file=path,
    filename="valores_reais.csv",
    initial_comment="Segue anexo do arquivo CSV com os valores reais convertidos diario:"
)

In [0]:
df_real_valor = ps.read_csv("dbfs:/databricks-results/prata/valores_reais/")

In [0]:
df_real_valor.columns[1:]

In [0]:
# Garante que o diretório exista no DBFS
output_dir = "/dbfs/imagens"
os.makedirs(output_dir, exist_ok=True)

for moeda in df_real_valor.columns[1:]:
    plt.figure(figsize=(10, 5))
    plt.plot(df_real_valor["data"], df_real_valor[moeda], marker='o')
    plt.title(f"Evolução de {moeda}")
    plt.xlabel("Data")
    plt.ylabel("Valor")
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    file_path = f"{output_dir}/{moeda}.png"
    plt.savefig(file_path)
    plt.close()
    
    print(f"Gráfico salvo em: {file_path}")

In [0]:
def send_images(moeda):
    
   send_image = client.files_upload_v2(
    channel="C096LMP8UJE",
    title="Enviando imagens de cotações",
    file=f'/dbfs/imagens/{moeda}.png',
    filename=f'{moeda}.png',
    initial_comment=f"Segue anexo do gráfico da evolução do valor do {moeda}:"
)

In [0]:
for moeda in df_real_valor.columns[1:]:
    send_images(moeda)